<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/CE_TRABALHO_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabalho Integrado Cognitive Enviroments
*   RM351578 - RODRIGO DE FEO VIEIRA
*   RM351070 - DANILO DOS SANTOS DA FONTE
*   RM350640 - FELIPE FIORIM CARVALHO SILVA
*   RM352054 - GABRIEL GONSALVES DE ALMEIDA


# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
from google.cloud import bigquery
from google.oauth2 import service_account


# Autenticação do Colab com o Bigquery

In [2]:
from google.colab import auth
auth.authenticate_user()

In [6]:
#@title Preenchimento do código do projeto
#@markdown Preencha abaixo o código do teu projeto na GCP. <br/>
#@markdown Após preenchimento do código, execute essa célula.   <br/>

project_id = "intense-wavelet-425818-j0"  #@param {type: "string"}
#@markdown ---
clientbq = bigquery.Client(project=project_id)

Teste de conectividade

In [37]:
%%time
query_sql = f"""
  SELECT COUNT( vendor_ID ) as VENDOR_ID, COUNT( DISTINCT VENDOR_ID ) as UNIQUE_VENDOR_ID
  FROM `{project_id}.cd_taxi_trip_2011.tb_taxi_trip_2011_5`
"""
query_job = clientbq.query(query_sql)  # Make an API request.
taxi_trip_dados = query_job.to_dataframe()
display(taxi_trip_dados)

,VENDOR_ID,UNIQUE_VENDOR_ID
0,28621842,2


CPU times: user 61.6 ms, sys: 5.27 ms, total: 66.8 ms
Wall time: 1.72 s


Vamos comecar a analisar a base de dados e responder as pergutas

– Em 2011, como foram as quantidades de corridas de acordo com o dia da semana? (0,5)
*   R: Conforme a distribuicao da quantidade de viagens durante os dias
dasemana o maior numero de corridas ocorreu na Sexta-Feira (4.3MM) e o menor numero de corridas foi na Segunda-Feira (3.6MM).

In [38]:
query_sql = f"""
  SELECT
      pickup_dt_day_week ,
      COUNT (*) qtd
  FROM
      (SELECT
          FORMAT_TIMESTAMP('%A', TIMESTAMP(PARSE_DATETIME('%m/%d/%Y %I:%M:%S %p', pickup_datetime))) AS pickup_dt_day_week,
       FROM
          `intense-wavelet-425818-j0.cd_taxi_trip_2011.tb_taxi_trip_2011_5` ) T
  GROUP BY
      pickup_dt_day_week
  ORDER BY
      qtd
"""
query_job = clientbq.query(query_sql)  # Make an API request.
taxi_trip_dados_r1 = query_job.to_dataframe()
display(taxi_trip_dados_r1)

,pickup_dt_day_week,qtd
0,Sunday,3635613
1,Monday,3777998
2,Tuesday,4152532
3,Saturday,4208881
4,Thursday,4275946
5,Wednesday,4276962
6,Friday,4293910


In [39]:
px.bar(taxi_trip_dados_r1.sort_values("pickup_dt_day_week"),
       y = "pickup_dt_day_week",
       x = "qtd",
       hover_data=["pickup_dt_day_week", "qtd"],
       color = "pickup_dt_day_week",
       title = "Quantidade de Viagens por Dia da Semana",
       labels={
        "pickup_dt_day_week": "Dia da Semana",
        "qtd": "Quantidade de Viagens"
         }
       )

In [40]:
px.pie(taxi_trip_dados_r1,
       values="qtd",
       names="pickup_dt_day_week")

- Em 2011, como foram os valores das corridas de acordo com o dia da semana? (0,5)

*   A Media de Valor por corrida foi maior na Quinta-Feira ( $1,03 ) e menor nos Sabados ($0,85).




In [59]:
query_sql = f"""
   SELECT
      pickup_dt_day_week ,
      AVG (total_amount) avg_total_amount
  FROM
      (SELECT
          FORMAT_TIMESTAMP('%A', TIMESTAMP(PARSE_DATETIME('%m/%d/%Y %I:%M:%S %p', pickup_datetime))) AS pickup_dt_day_week,
          SAFE_CAST(total_amount AS NUMERIC) as total_amount
       FROM
          `intense-wavelet-425818-j0.cd_taxi_trip_2011.tb_taxi_trip_2011_11` ) T
  GROUP BY
      pickup_dt_day_week
  ORDER BY
      avg_total_amount
"""
query_job = clientbq.query(query_sql)  # Make an API request.
taxi_trip_dados_r2 = query_job.to_dataframe()
display(taxi_trip_dados_r2)

,pickup_dt_day_week,avg_total_amount
0,Saturday,0.843304292
1,Sunday,0.918654857
2,Friday,0.976215482
3,Monday,0.978329554
4,Tuesday,0.995812740
5,Wednesday,1.015533673
6,Thursday,1.028710999


In [60]:
px.bar(taxi_trip_dados_r2.sort_values("pickup_dt_day_week"),
       y = "avg_total_amount",
       x = "pickup_dt_day_week",
       hover_data=["pickup_dt_day_week", "avg_total_amount"],
       color = "pickup_dt_day_week",
       title = "Média do Valor Total da Viagem por Dia da Semana",
       labels={
        "pickup_dt_day_week": "Dia da Semana",
        "avg_total_amount": "Media do Valor Total da Viagem"
         }
       )

In [61]:
px.pie(taxi_trip_dados_r2,
       values="avg_total_amount",
       names="pickup_dt_day_week")

-Em 2011, como foram os valores das corridas de acordo com a quantidade de passageiros? (0,25)

*  R: O valor medio das corridas com 10 passageiros e o Maior e as corridas com 4 pessoas sao os mais baixos


In [84]:
query_sql = f"""
   SELECT
      SAFE_CAST(passenger_count AS STRING ) passenger_count,
      avg_total_amount
 FROM

 (SELECT
      passenger_count ,
      AVG (total_amount) avg_total_amount
  FROM
      (SELECT
          SAFE_CAST(passenger_count AS INTEGER) as passenger_count,
          SAFE_CAST(total_amount AS NUMERIC) as total_amount
       FROM
          `intense-wavelet-425818-j0.cd_taxi_trip_2011.tb_taxi_trip_2011_11` ) T

  GROUP BY
      passenger_count
  ORDER BY
      passenger_count ) G
 WHERE
      G.avg_total_amount <> 0 AND G.passenger_count <> 0
"""
query_job = clientbq.query(query_sql)  # Make an API request.
taxi_trip_dados_r3 = query_job.to_dataframe()
display(taxi_trip_dados_r3)

,passenger_count,avg_total_amount
0,1,0.998640692
1,2,0.918763860
2,3,0.810715679
3,4,0.686163673
4,5,0.973862828
5,6,0.999409508
6,7,0.787500000
7,9,2.000000000
8,10,7.000000000


In [87]:
px.bar(taxi_trip_dados_r3.sort_values("passenger_count"),
       y = "avg_total_amount",
       x = "passenger_count",
       hover_data=["avg_total_amount","passenger_count"],
       color = "avg_total_amount",
       title = "Média do Valor Total da Viagem por Quantidade de Passageiros",
       labels={
        "passenger_count": "Quatidade de Passageiros",
        "avg_total_amount": "Media do Valor Total da Viagem"
         }
       )

In [75]:
px.pie(taxi_trip_dados_r3,
       values="avg_total_amount",
       names="passenger_count")